##### Análise de desempenho de modelos

In [1]:
import pandas as pd
import sys
import os
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler, MaxAbsScaler
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier

In [2]:
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

In [3]:
from avaliador import funcao_avaliadora

In [4]:
df = pd.read_csv("../data/dados_transformados.csv")

In [5]:
X = df.drop(['HeartDisease', 'RestingECG_ST', 'RestingECG_Normal', 'ChestPainType_TA', 'Cholesterol'], axis=1)
y = df['HeartDisease']

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

In [7]:
modelos = {
    "LogisticRegression": LogisticRegression(),
    "DecisionTreeClassifier": DecisionTreeClassifier(),
    "RandomForestClassifier": RandomForestClassifier(),
    "GradientBoostingClassifier": GradientBoostingClassifier(),
    "XGBClassifier": XGBClassifier(),
}

In [8]:
melhores_modelos = {
    'accuracy': {'nome': None, 'valor': 0},
    'precision': {'nome': None, 'valor': 0},
    'recall': {'nome': None, 'valor': 0},
    'f1': {'nome': None, 'valor': 0},
    'roc_auc': {'nome': None, 'valor': 0},
    'mcc': {'nome': None, 'valor': 0},
}

In [9]:
scalers = [
    ('RobustScaler', RobustScaler()),
    ('StandardScaler', StandardScaler()),
    ('MinMaxScaler', MinMaxScaler()),
    ('MaxAbsScaler', MaxAbsScaler())
]

In [10]:
for nome_scaler, scaler in scalers:
    for nome, modelo in modelos.items():
        pipeline = Pipeline([
            ('scaler', scaler),
            ('pca', PCA(n_components=4)),
            ('feature_selection', SelectKBest(score_func=f_classif, k='all')),
            ('modelo', modelo)
        ])

        resultados = funcao_avaliadora(pipeline, X_train, X_test, y_train, y_test)

        accuracy_test = resultados["Classification Report Test"]['accuracy']
        precision_test = resultados["Classification Report Test"]['weighted avg']['precision']
        recall_test = resultados["Classification Report Test"]['weighted avg']['recall']
        f1_test = resultados["Classification Report Test"]['weighted avg']['f1-score']
        roc_auc_test = resultados["ROC AUC Test"]
        mcc_test = resultados["Matthews Correlation Coefficient Test"]

        print(f'\n{nome} com {nome_scaler}:')
        print(f'Accuracy: {accuracy_test}')
        print(f'Precision: {precision_test}')
        print(f'Recall: {recall_test}')
        print(f'F1 Score: {f1_test}')
        print(f'ROC AUC: {roc_auc_test}')
        print(f'Matthews Correlation Coefficient: {mcc_test}\n')
        print(f'Confusion Matrix: \n{resultados["Confusion Matrix Test"]}')
        print('-' * 60)

        if accuracy_test > melhores_modelos['accuracy']['valor']:
            melhores_modelos['accuracy']['nome'] = f'{nome} com {nome_scaler}'
            melhores_modelos['accuracy']['valor'] = accuracy_test
        
        if precision_test > melhores_modelos['precision']['valor']:
            melhores_modelos['precision']['nome'] = f'{nome} com {nome_scaler}'
            melhores_modelos['precision']['valor'] = precision_test
        
        if recall_test > melhores_modelos['recall']['valor']:
            melhores_modelos['recall']['nome'] = f'{nome} com {nome_scaler}'
            melhores_modelos['recall']['valor'] = recall_test
        
        if f1_test > melhores_modelos['f1']['valor']:
            melhores_modelos['f1']['nome'] = f'{nome} com {nome_scaler}'
            melhores_modelos['f1']['valor'] = f1_test

        if roc_auc_test > melhores_modelos['roc_auc']['valor']:
            melhores_modelos['roc_auc']['nome'] = f'{nome} com {nome_scaler}'
            melhores_modelos['roc_auc']['valor'] = roc_auc_test

        if mcc_test > melhores_modelos['mcc']['valor']:
            melhores_modelos['mcc']['nome'] = f'{nome} com {nome_scaler}'
            melhores_modelos['mcc']['valor'] = mcc_test

print("\nMelhores modelos:")
print(f'Melhor modelo para Accuracy: {melhores_modelos["accuracy"]["nome"]} com valor {melhores_modelos["accuracy"]["valor"]}')
print(f'Melhor modelo para Precision: {melhores_modelos["precision"]["nome"]} com valor {melhores_modelos["precision"]["valor"]}')
print(f'Melhor modelo para Recall: {melhores_modelos["recall"]["nome"]} com valor {melhores_modelos["recall"]["valor"]}')
print(f'Melhor modelo para F1 Score: {melhores_modelos["f1"]["nome"]} com valor {melhores_modelos["f1"]["valor"]}')
print(f'Melhor modelo para ROC AUC: {melhores_modelos["roc_auc"]["nome"]} com valor {melhores_modelos["roc_auc"]["valor"]}')
print(f'Melhor modelo para MCC: {melhores_modelos["mcc"]["nome"]} com valor {melhores_modelos["mcc"]["valor"]}')


LogisticRegression com RobustScaler:
Accuracy: 0.86
Precision: 0.8599893029060439
Recall: 0.86
F1 Score: 0.8599563493831011
ROC AUC: 0.9375
Matthews Correlation Coefficient: 0.719465618737283

Confusion Matrix: 
[[61 11]
 [10 68]]
------------------------------------------------------------

DecisionTreeClassifier com RobustScaler:
Accuracy: 0.7866666666666666
Precision: 0.7870981507823612
Recall: 0.7866666666666666
F1 Score: 0.7867425724959972
ROC AUC: 0.7868589743589743
Matthews Correlation Coefficient: 0.573309753754341

Confusion Matrix: 
[[57 15]
 [17 61]]
------------------------------------------------------------

RandomForestClassifier com RobustScaler:
Accuracy: 0.8466666666666667
Precision: 0.8468101761252446
Recall: 0.8466666666666667
F1 Score: 0.8467007786429366
ROC AUC: 0.9153311965811967
Matthews Correlation Coefficient: 0.6930676133279396

Confusion Matrix: 
[[61 11]
 [12 66]]
------------------------------------------------------------

GradientBoostingClassifier com 